In [ ]:
sample_df = df.sample(n=100000)

# write the sample DataFrame to a CSV file
sample_df.to_csv('100000twtunlabeled.csv', index=False)

# confirm that the new file was created and contains the expected data
new_df = pd.read_csv('100000twtunlabeled.csv')
print(new_df.head())

In [ ]:
test_data = pd.read_csv('100000twtfinance.csv', sep=',')

In [ ]:
test_data = pd.read_csv('100000twtnotfinance.csv', sep=',')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.3 MB/s eta 0:00:00


In [ ]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 10.1 MB/s eta 0:00:00


In [ ]:
import os
file_path = '/content/drive/MyDrive/Colab Notebooks/tlg_labeled.pth'
print(os.path.isfile(file_path))


False


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.4 MB/s eta 0:00:00


In [ ]:
import chardet

# Read the CSV file as binary
with open('5000_tlg_labeled.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Print the detected encoding
print(result['encoding'])

Windows-1252


In [4]:
from google.colab import files


uploaded = files.upload()


Saving 300_tlg_unlabeled.csv to 300_tlg_unlabeled.csv


In [ ]:
import os
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup

# Load the labeled training set and create input tensors
train_data = pd.read_csv('5000_tlg_labeled.csv', sep=';', encoding='Windows-1252')

# Filter the training data to include only 500 rows for each label
train_data = train_data.groupby('label').head(500)

# Convert labels to integers
label2int = {"J": 0, "M": 1, "T": 2}
train_data['label'] = train_data['label'].map(label2int)

# Get the training texts and labels as lists
train_texts = train_data["content"].tolist()
train_labels = train_data["label"].tolist()

# Load the BERT tokenizer and tokenize the input texts
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt")

# Create a PyTorch dataset from the input encodings and labels
train_dataset = TensorDataset(
    train_encodings["input_ids"],
    train_encodings["attention_mask"],
    torch.tensor(train_labels),
)

# Load the BERT model and set the device
model = BertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-cased', num_labels=3)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Define the training hyperparameters
batch_size = 2
learning_rate = 1e-5
num_epochs = 2
warmup_steps = 0.1 * (len(train_dataset) // batch_size)
total_steps = len(train_dataset) // batch_size * num_epochs
num_warmup_steps = int(warmup_steps)
num_training_steps = int(total_steps - num_warmup_steps)

# Create a PyTorch DataLoader for the training set
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Create the optimizer and the scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Define the loss function and the evaluation metric
loss_fn = torch.nn.CrossEntropyLoss()
metric_fn = torch.nn.functional.softmax

# Training loop
model.train()

for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        batch_input_ids = batch[0].to(device)
        batch_attention_mask = batch[1].to(device)
        batch_labels = batch[2].to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits

        # Calculate loss
        loss = loss_fn(logits, batch_labels)

        # Backward pass
        loss.backward()

        # Clip the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update the parameters
        optimizer.step()

        # Update the learning rate scheduler
        scheduler.step()

        # Print the loss for every 10th batch
        if step % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Step {step+1}/{len(train_dataloader)}, Loss {loss.item():.4f}")

# Save the fine-tuned model
model_path = "/content/drive/MyDrive/ColabNotebookModels/tlg_labeled.pth"

if os.path.exists(model_path):
    os.remove(model_path)

torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

Epoch 1/2, Step 1/450, Loss 0.9527
Epoch 1/2, Step 11/450, Loss 1.3660
Epoch 1/2, Step 21/450, Loss 1.3442
Epoch 1/2, Step 31/450, Loss 0.6865
Epoch 1/2, Step 41/450, Loss 1.3360
Epoch 1/2, Step 51/450, Loss 1.0298
Epoch 1/2, Step 61/450, Loss 1.2141
Epoch 1/2, Step 71/450, Loss 1.2051
Epoch 1/2, Step 81/450, Loss 1.2652
Epoch 1/2, Step 91/450, Loss 0.7484
Epoch 1/2, Step 101/450, Loss 0.9562
Epoch 1/2, Step 111/450, Loss 0.9988
Epoch 1/2, Step 121/450, Loss 1.0890
Epoch 1/2, Step 131/450, Loss 0.6198
Epoch 1/2, Step 141/450, Loss 0.6485
Epoch 1/2, Step 151/450, Loss 0.5769
Epoch 1/2, Step 161/450, Loss 1.5471
Epoch 1/2, Step 171/450, Loss 1.2931
Epoch 1/2, Step 181/450, Loss 1.0751
Epoch 1/2, Step 191/450, Loss 0.5123
Epoch 1/2, Step 201/450, Loss 0.7364
Epoch 1/2, Step 211/450, Loss 1.3563
Epoch 1/2, Step 221/450, Loss 0.2423
Epoch 1/2, Step 231/450, Loss 0.3775
Epoch 1/2, Step 241/450, Loss 1.2729
Epoch 1/2, Step 251/450, Loss 0.8302
Epoch 1/2, Step 261/450, Loss 0.8841
Epoch 1/2, S

In [ ]:
import os
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Load the labeled training set and create input tensors
train_data = pd.read_csv('5000_tlg_labeled.csv', sep=';', encoding='Windows-1252')

# Filter the training data to include only 500 rows for each label
train_data = train_data.groupby('label').head(500)

# Convert labels to integers
label2int = {"J": 0, "M": 1, "T": 2}
train_data['label'] = train_data['label'].map(label2int)

# Get the training texts and labels as lists
# # Usename
train_texts = train_data["content"].tolist()
train_labels = train_data["label"].tolist()

# Load the BERT tokenizer and tokenize the input texts
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt")

# Create a PyTorch dataset from the input encodings and labels
train_dataset = TensorDataset(
    train_encodings["input_ids"],
    train_encodings["attention_mask"],
    torch.tensor(train_labels),
)

# Define the training hyperparameters
batch_size = 2
learning_rate = 1e-5
num_epochs = 2
warmup_steps = 0.1 * (len(train_dataset) // batch_size)
total_steps = len(train_dataset) // batch_size * num_epochs
num_warmup_steps = int(warmup_steps)
num_training_steps = int(total_steps - num_warmup_steps)

# Define evaluation metrics
def evaluate(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in dataloader:
            batch_input_ids = batch[0].to(device)
            batch_attention_mask = batch[1].to(device)
            batch_labels = batch[2].to(device)

            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            all_labels.extend(batch_labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    confusion = confusion_matrix(all_labels, all_preds)
    class_report = classification_report(all_labels, all_preds)

    return accuracy, f1, precision, recall, confusion, class_report

# Fine-tuning loop
for iteration in range(5):  # Run for 5 iterations or adjust as needed
    # Load the BERT model and set the device
    model = BertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-cased', num_labels=3)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    # Create a PyTorch DataLoader for the training set
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

    # Create the optimizer and the scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

    # Training loop
    model.train()
    for epoch in range(num_epochs):
        for step, batch in enumerate(train_dataloader):
            batch_input_ids = batch[0].to(device)
            batch_attention_mask = batch[1].to(device)
            batch_labels = batch[2].to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits

            # Calculate loss
            loss = loss_fn(logits, batch_labels)

            # Backward pass
            loss.backward()

            # Clip the gradients to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update the parameters
            optimizer.step()

            # Update the learning rate scheduler
            scheduler.step()

            # Print the loss for every 10th batch
            if step % 10 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Step {step+1}/{len(train_dataloader)}, Loss {loss.item():.4f}")

    # Evaluate the model on the training set
    accuracy, f1, precision, recall, confusion, class_report = evaluate(model, train_dataloader)
    print(f"Iteration {iteration+1}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(class_report)

    # Save the fine-tuned model
    model_path = f"/content/drive/MyDrive/ColabNotebookModels/tlg_labeled.pth"

    if os.path.exists(model_path):
        os.remove(model_path)

    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")


Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

Epoch 1/2, Step 1/450, Loss 1.2477
Epoch 1/2, Step 11/450, Loss 0.7871
Epoch 1/2, Step 21/450, Loss 1.1926
Epoch 1/2, Step 31/450, Loss 1.1322
Epoch 1/2, Step 41/450, Loss 0.7180
Epoch 1/2, Step 51/450, Loss 1.1442
Epoch 1/2, Step 61/450, Loss 0.8694
Epoch 1/2, Step 71/450, Loss 1.1660
Epoch 1/2, Step 81/450, Loss 1.0880
Epoch 1/2, Step 91/450, Loss 0.9828
Epoch 1/2, Step 101/450, Loss 1.3570
Epoch 1/2, Step 111/450, Loss 0.9899
Epoch 1/2, Step 121/450, Loss 0.9856
Epoch 1/2, Step 131/450, Loss 1.0387
Epoch 1/2, Step 141/450, Loss 1.2086
Epoch 1/2, Step 151/450, Loss 0.8434
Epoch 1/2, Step 161/450, Loss 1.0908
Epoch 1/2, Step 171/450, Loss 0.5340
Epoch 1/2, Step 181/450, Loss 0.8464
Epoch 1/2, Step 191/450, Loss 0.6039
Epoch 1/2, Step 201/450, Loss 0.3896
Epoch 1/2, Step 211/450, Loss 0.8284
Epoch 1/2, Step 221/450, Loss 0.8413
Epoch 1/2, Step 231/450, Loss 1.1744
Epoch 1/2, Step 241/450, Loss 0.4425
Epoch 1/2, Step 251/450, Loss 0.7877
Epoch 1/2, Step 261/450, Loss 0.9778
Epoch 1/2, S

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

Epoch 1/2, Step 1/450, Loss 1.2883
Epoch 1/2, Step 11/450, Loss 1.0810
Epoch 1/2, Step 21/450, Loss 1.5284
Epoch 1/2, Step 31/450, Loss 0.9493
Epoch 1/2, Step 41/450, Loss 1.1963
Epoch 1/2, Step 51/450, Loss 1.1332
Epoch 1/2, Step 61/450, Loss 1.0705
Epoch 1/2, Step 71/450, Loss 0.8271
Epoch 1/2, Step 81/450, Loss 0.8493
Epoch 1/2, Step 91/450, Loss 0.9387
Epoch 1/2, Step 101/450, Loss 1.2575
Epoch 1/2, Step 111/450, Loss 0.9174
Epoch 1/2, Step 121/450, Loss 0.7264
Epoch 1/2, Step 131/450, Loss 1.0316
Epoch 1/2, Step 141/450, Loss 0.7504
Epoch 1/2, Step 151/450, Loss 0.9110
Epoch 1/2, Step 161/450, Loss 0.7638
Epoch 1/2, Step 171/450, Loss 0.9163
Epoch 1/2, Step 181/450, Loss 0.7641
Epoch 1/2, Step 191/450, Loss 1.3461
Epoch 1/2, Step 201/450, Loss 1.3878
Epoch 1/2, Step 211/450, Loss 1.4889
Epoch 1/2, Step 221/450, Loss 0.3106
Epoch 1/2, Step 231/450, Loss 1.3021
Epoch 1/2, Step 241/450, Loss 1.3629
Epoch 1/2, Step 251/450, Loss 1.1143
Epoch 1/2, Step 261/450, Loss 0.5651
Epoch 1/2, S

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

Epoch 1/2, Step 1/450, Loss 1.1682
Epoch 1/2, Step 11/450, Loss 1.3990
Epoch 1/2, Step 21/450, Loss 1.1430
Epoch 1/2, Step 31/450, Loss 1.6449
Epoch 1/2, Step 41/450, Loss 1.0382
Epoch 1/2, Step 51/450, Loss 0.9200
Epoch 1/2, Step 61/450, Loss 1.1947
Epoch 1/2, Step 71/450, Loss 0.7641
Epoch 1/2, Step 81/450, Loss 1.0639
Epoch 1/2, Step 91/450, Loss 1.3055
Epoch 1/2, Step 101/450, Loss 1.0860
Epoch 1/2, Step 111/450, Loss 0.8838
Epoch 1/2, Step 121/450, Loss 0.8907
Epoch 1/2, Step 131/450, Loss 0.7868
Epoch 1/2, Step 141/450, Loss 0.9901
Epoch 1/2, Step 151/450, Loss 0.7894
Epoch 1/2, Step 161/450, Loss 0.6920
Epoch 1/2, Step 171/450, Loss 1.0520
Epoch 1/2, Step 181/450, Loss 0.7689
Epoch 1/2, Step 191/450, Loss 0.9385
Epoch 1/2, Step 201/450, Loss 1.2912
Epoch 1/2, Step 211/450, Loss 0.6366
Epoch 1/2, Step 221/450, Loss 0.6430
Epoch 1/2, Step 231/450, Loss 0.9653
Epoch 1/2, Step 241/450, Loss 1.3406
Epoch 1/2, Step 251/450, Loss 0.7574
Epoch 1/2, Step 261/450, Loss 0.5314
Epoch 1/2, S

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

Epoch 1/2, Step 1/450, Loss 1.2075
Epoch 1/2, Step 11/450, Loss 1.4120
Epoch 1/2, Step 21/450, Loss 1.0815
Epoch 1/2, Step 31/450, Loss 1.5424
Epoch 1/2, Step 41/450, Loss 1.4132
Epoch 1/2, Step 51/450, Loss 0.6305
Epoch 1/2, Step 61/450, Loss 1.6301
Epoch 1/2, Step 71/450, Loss 1.2726
Epoch 1/2, Step 81/450, Loss 1.0484
Epoch 1/2, Step 91/450, Loss 0.9549
Epoch 1/2, Step 101/450, Loss 0.9500
Epoch 1/2, Step 111/450, Loss 1.0574
Epoch 1/2, Step 121/450, Loss 0.9194
Epoch 1/2, Step 131/450, Loss 1.4121
Epoch 1/2, Step 141/450, Loss 0.8272
Epoch 1/2, Step 151/450, Loss 1.2300
Epoch 1/2, Step 161/450, Loss 0.8862
Epoch 1/2, Step 171/450, Loss 0.6184
Epoch 1/2, Step 181/450, Loss 0.4291
Epoch 1/2, Step 191/450, Loss 1.3879
Epoch 1/2, Step 201/450, Loss 1.1234
Epoch 1/2, Step 211/450, Loss 1.9287
Epoch 1/2, Step 221/450, Loss 0.9156
Epoch 1/2, Step 231/450, Loss 0.5548
Epoch 1/2, Step 241/450, Loss 1.0504
Epoch 1/2, Step 251/450, Loss 0.6196
Epoch 1/2, Step 261/450, Loss 1.4290
Epoch 1/2, S

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

Epoch 1/2, Step 1/450, Loss 1.0202
Epoch 1/2, Step 11/450, Loss 0.9458
Epoch 1/2, Step 21/450, Loss 1.5173
Epoch 1/2, Step 31/450, Loss 1.3321
Epoch 1/2, Step 41/450, Loss 1.1664
Epoch 1/2, Step 51/450, Loss 1.3726
Epoch 1/2, Step 61/450, Loss 1.0700
Epoch 1/2, Step 71/450, Loss 0.9924
Epoch 1/2, Step 81/450, Loss 1.0529
Epoch 1/2, Step 91/450, Loss 0.9359
Epoch 1/2, Step 101/450, Loss 1.2826
Epoch 1/2, Step 111/450, Loss 0.7261
Epoch 1/2, Step 121/450, Loss 0.9627
Epoch 1/2, Step 131/450, Loss 1.2056
Epoch 1/2, Step 141/450, Loss 0.6503
Epoch 1/2, Step 151/450, Loss 1.0256
Epoch 1/2, Step 161/450, Loss 1.0969
Epoch 1/2, Step 171/450, Loss 0.5540
Epoch 1/2, Step 181/450, Loss 1.6916
Epoch 1/2, Step 191/450, Loss 1.1236
Epoch 1/2, Step 201/450, Loss 0.9314
Epoch 1/2, Step 211/450, Loss 1.7279
Epoch 1/2, Step 221/450, Loss 1.4731
Epoch 1/2, Step 231/450, Loss 0.7381
Epoch 1/2, Step 241/450, Loss 0.2104
Epoch 1/2, Step 251/450, Loss 1.0145
Epoch 1/2, Step 261/450, Loss 0.7249
Epoch 1/2, S

In [5]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd

# Load the retrained model
model = BertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-cased', num_labels=3)
model.load_state_dict(torch.load("/content/drive/MyDrive/ColabNotebookModels/tlg_labeled.pth"))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Load the test set and create input tensors
# Assume that the test data is in CSV format with two columns: message_detail and username
test_data = pd.read_csv('300_tlg_unlabeled.csv', sep=',')
test_texts = test_data["content"].tolist()

# Load the BERT tokenizer and tokenize the input texts
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')

test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")
test_dataset = torch.utils.data.TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
)

# Use the trained model to make predictions on the test set
model.eval()
predictions = []
for batch in torch.utils.data.DataLoader(test_dataset, batch_size=8):
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        outputs = model(**inputs)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, axis=1)
        predictions.extend(batch_predictions.detach().cpu().numpy().tolist())

# Add the predicted labels to the test data DataFrame
test_data["predicted_label"] = predictions

# Save the labeled dataset to a CSV file
test_data.to_csv("/content/drive/MyDrive/Colab Notebooks/tlg_labeled_predictions.csv", index=False)
print("Labeled dataset saved to tlg_labeled_predictions.csv")

# Print the Text and the predicted label of each example in the test set
for i, text in enumerate(test_texts):
    label = predictions[i]
    print(f"{text}\n{label}\n")


Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

Labeled dataset saved to tlg_labeled_predictions.csv
Saat ⏰ 11 .00 de Giriyoruz hep beraber Nakit 💴 hazırla
0

#EREGL 31.16 YÜRÜ BEEEE 😁 😁 💪 💪 💪 💪 💪
1

DOSTLAR BOMBA 💣 GİBİ BİR HİSSE SİZLER VERİYORUM ŞİMDİ HAZIR OLALIM.. 💪 💪 💪
0

#BIZIM 🌻 Tavan olmadı mı? Çabuk ol 👈 💴 💴 💴 💴
1

Arkadaşlar ... Zor bir gün oldu Endeksi hacmi çok düşüktü Gün içinde haber akışı biraz kötü olunca piyasa bozdu Sonra 15 dk endeksi iyi topladılar Bu gün 2000 puan üzeri ve (+) kapanış güzel Haftaya güzel olacak .... Sakin ve yolumuza devam İyi akşamlar
2

#fener de 31 e düşmicek bence
1

Gıdalar ve marketler gidecek ... 1/2 gün içinde bir anlaşma 🤝 olcak Rusya ufak bir gösteri 👈 Sakın unutma Bu sadece bir oyun 👈 Malını kaptırma
2

#fener 30 u alırsa tamamdır
1

BİST TOPARLANIR.. 💪 Durmak yok yola devam.. 💪
0

Bu gün.. Gıdalar çok önemli!!! #DARDL 🐟 Topla 👈 #BIZIM 🌻 Topla 👈 #MERKO 🍫 Topla 👈 Sonra #ULUUN 🌾 #ICBCT 🇨🇳 #DGGYO 🚢 Öncelik.... Dardanel ve bizim
1

#yyapı tavana 1 milyon TL kaldı Ben ne dedim size
1

#BIZ

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print()
